# 1. Neural Network Training

In [1]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler

from Ranker.neuralnetwork import NeuralNetwork
from DataHandling.train_data import load_data


### 1.1. Load training and test data

In [2]:
feature_indices = [108, 18, 103, 3, 8, 13, 125, 126, 107, 17, 102, 2, 7, 12, 105, 15, 100, 0, 5, 10]

In [3]:
# load training data
X_train, y_train = load_data(path="../../../data/MSLR-WEB10K/Fold1/train.txt", nrows=700000, feature_indices=feature_indices)
#X_test, y_test = load_data(path="../../../data/MSLR-WEB10K/Fold1/test.txt", nrows=10000)


In [4]:
print('label counts', np.unique(y_train, return_counts=True))

label counts (array([0., 1., 2., 3., 4.]), array([367404, 224103,  91497,  12066,   4930]))


In [11]:
# balance dataset
indices = np.empty(1, dtype=int)
for label in range(5):
    indices = np.append(indices, np.random.choice(np.where(y_train == label)[0], 4930, replace=False))
    


X_cut = X_train[indices, :]
y_cut = y_train[indices]

print(X_cut.shape, y_cut.shape)
print('label counts: ', np.unique(y_cut, return_counts=True))

(24651, 20) (24651,)
label counts:  (array([0., 1., 2., 3., 4.]), array([4931, 4930, 4930, 4930, 4930]))


In [12]:
y_trans = torch.zeros(len(y_cut), 5)
for i, label in enumerate(y_cut):
    y_trans[i, 0:int(label)+1] = 1 


### 1.2. Create model

In [13]:
model = NeuralNetwork(n_features=len(feature_indices), n_hidden=10)

In [14]:
### define optimizer and loss fct
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss = nn.CrossEntropyLoss()

### 1.3. Train Model

In [15]:
for epoch in range(10):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = model.train_loop(torch.Tensor(X_cut), y_trans, loss, optimizer)

Epoch 1
-------------------------------
loss: 44.577934  [   32/24651]
loss: 5.189493  [16032/24651]
Epoch 2
-------------------------------
loss: 4.246328  [   32/24651]
loss: 4.613749  [16032/24651]
Epoch 3
-------------------------------
loss: 4.876898  [   32/24651]
loss: 4.477571  [16032/24651]
Epoch 4
-------------------------------
loss: 4.881222  [   32/24651]
loss: 3.918021  [16032/24651]
Epoch 5
-------------------------------
loss: 4.515180  [   32/24651]
loss: 4.397859  [16032/24651]
Epoch 6
-------------------------------
loss: 4.279982  [   32/24651]
loss: 4.154069  [16032/24651]
Epoch 7
-------------------------------
loss: 4.316090  [   32/24651]
loss: 4.458076  [16032/24651]
Epoch 8
-------------------------------
loss: 4.539814  [   32/24651]
loss: 4.019837  [16032/24651]
Epoch 9
-------------------------------
loss: 4.301822  [   32/24651]
loss: 4.900565  [16032/24651]
Epoch 10
-------------------------------
loss: 4.258908  [   32/24651]
loss: 4.719433  [16032/24651

### 1.4. Save Model

In [10]:
# save mdoel
model.save("../models/nn.pth")

model saved in  ../models/nn.pth
